In [1]:
import os
import sys
sys.path.append(os.getcwd()+'/..')

from pdearena import utils
import torch
from pdearena.data.datamodule import PDEDataModule
from pdearena.lr_scheduler import LinearWarmupCosineAnnealingLR  # noqa: F401
from pdearena.models.pdemodel import PDEModel

from helper_functions.data import write_h5, read_h5_numpy
from helper_functions.general import touch_dir
from time import time
import numpy as np
import h5py
from tqdm import tqdm
import shutil
import _pickle as cPickle
import pickle 
import hdf5plugin


In [3]:
import sys
sys.path


['/home/bonotto/RESEARCH/pdearena/pdedatagen',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python38.zip',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8/lib-dynload',
 '',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8/site-packages',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/ENI/Projects/Prove/sci_ML/pdearena',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/ENI/Projects/Prove/helper_functions',
 '/home/bonotto/RESEARCH/pdearena/pdedatagen/..',
 '/tmp/tmplccq5hr1']

In [4]:
def hdf5_to_dict(file):
    d = dict()
    for key,val in file.items():
        if type(val) == h5py._hl.dataset.Dataset:
            d[key] = np.array(val)
        else:
            d[key] = hdf5_to_dict(val)
    return d

In [5]:
### Chosen the following saving strategies
'''
- hdf5plugin.Blosc(cname='lz4', clevel = 1)
- pickle
'''
compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)

input_folder = '../pdearena_data/NavierStokes-2D-32x32/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-32x32_pkl/',
]
files = os.listdir(input_folder)


def convert_file(filein,fileout):
    with h5py.File(filein, 'r') as hf_source:
        d = hdf5_to_dict(hf_source)
        if 'pkl' in fileout:
            with open(fileout,"wb") as filehandler:
                cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)
            # pass
        else:
            pass
            # with h5py.File(fileout, "w") as h5f_target:
            #     mode = next(iter(d.keys()))
            #     dataset = h5f_target.create_group(mode)
            #     for k,v in d[mode].items():
            #         # print('processing {}'.format(k))
            #         dataset.create_dataset(
            #             k, 
            #             data = v,
            #             shape = v.shape,
            #             **compressor
            #         )


for target_folder in target_folders:
    touch_dir(target_folder)
    
    for file in tqdm(files,total=len(files)):
        try:
            if 'yaml' not in file:
                fileout = file
                if 'pkl' in target_folder:
                    fileout = fileout[:-3] + '.pkl'
                convert_file(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
            else:
                fileout = file
                shutil.copyfile(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
        except:
            print(file,fileout)
        



100%|██████████| 310/310 [00:06<00:00, 45.11it/s]


In [8]:
def load_pickle(filename):
    with open(filename,'rb') as filehandler:
        return cPickle.load(filehandler)

def save_pickle(data, filename):
    with open(filename,"wb") as filehandler:
        cPickle.dump(data,filehandler,protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
### Save 1 training sample per pickle

compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)

input_folder = '../pdearena_data/NavierStokes-2D-32x32_pkl/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-32x32_pkl_single/',
]

input_folder = '../pdearena_data/NavierStokes-2D-64x64_pkl/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-64x64_pkl_single/',
]
files = os.listdir(input_folder)

here = os.getcwd()


# def convert_file(filein,fileout):
#     with h5py.File(filein, 'r') as hf_source:
#         d = hdf5_to_dict(hf_source)
#         if 'pkl' in fileout:
#             with open(fileout,"wb") as filehandler:
#                 cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)
#             # pass
#         else:
#             pass
#             # with h5py.File(fileout, "w") as h5f_target:
#             #     mode = next(iter(d.keys()))
#             #     dataset = h5f_target.create_group(mode)
#             #     for k,v in d[mode].items():
#             #         # print('processing {}'.format(k))
#             #         dataset.create_dataset(
#             #             k, 
#             #             data = v,
#             #             shape = v.shape,
#             #             **compressor
#             #         )


for target_folder in target_folders:
    touch_dir(target_folder)
    
    for file in tqdm(files,total=len(files)):
        try:
            if 'yaml' not in file:

                d = load_pickle(
                    os.path.join(input_folder,file)
                )

                mode = next(iter(d.keys()))

                for i in range(d[mode]['u'].shape[0]):
                    d_i = {}
                    d_i[mode] = {}

                    for k in d[mode].keys():
                        d_i[mode][k] = d[mode][k][i,...]
                        d_i[mode][k] = d_i[mode][k][np.newaxis,...] 
                        # print(k,d[mode][k].shape)
                        # print(k,d_i[mode][k].shape)

                    file_i = file.replace('.pkl','') + '_' + str(i) + '.pkl'
                    file_i = os.path.join(target_folder,file_i)
                    file_i = os.path.join(
                        os.getcwd(),
                        file_i
                    )
                    with open(file_i,"wb") as filehandler:
                        cPickle.dump(d_i,filehandler,protocol=pickle.HIGHEST_PROTOCOL)

        except:
            print(file,fileout)
        



  0%|          | 1/310 [00:16<1:25:19, 16.57s/it]

buo_y (25,)
dt (25,)
dx (25,)
dy (25,)
t (25, 14)
u (25, 14, 32, 32)
vx (25, 14, 32, 32)
vy (25, 14, 32, 32)
x (25, 32)
y (25, 32)
buo_y (25,)
buo_y ()
dt (25,)
dt ()
dx (25,)
dx ()
dy (25,)
dy ()
t (25, 14)
t (14,)
u (25, 14, 32, 32)
u (14, 32, 32)
vx (25, 14, 32, 32)
vx (14, 32, 32)
vy (25, 14, 32, 32)
vy (14, 32, 32)
x (25, 32)
x (32,)
y (25, 32)
y (32,)
buo_y ()
dt ()
dx ()
dy ()
t (14,)
u (14, 32, 32)
vx (14, 32, 32)
vy (14, 32, 32)
x (32,)
y (32,)
buo_y (1,)
dt (1,)
dx (1,)
dy (1,)
t (1, 14)
u (1, 14, 32, 32)
vx (1, 14, 32, 32)
vy (1, 14, 32, 32)
x (1, 32)
y (1, 32)


 17%|█▋        | 54/310 [22:58<1:17:59, 18.28s/it] 

pde_train_seed_429673.yaml NavierStokes2D_valid_142203_0.50000..pkl
NavierStokes2D_train_392020_0.50000_100.pkl NavierStokes2D_train_392020_0.50000_100..pkl
NavierStokes2D_train_857042_0.50000_100.pkl NavierStokes2D_train_857042_0.50000_100..pkl
NavierStokes2D_train_891593_0.50000_100.pkl NavierStokes2D_train_891593_0.50000_100..pkl
NavierStokes2D_valid_975633_0.50000.pkl NavierStokes2D_valid_975633_0.50000..pkl
NavierStokes2D_valid_337493_0.50000.pkl NavierStokes2D_valid_337493_0.50000..pkl
NavierStokes2D_valid_666697_0.50000.pkl NavierStokes2D_valid_666697_0.50000..pkl
NavierStokes2D_test_445836_0.50000.pkl NavierStokes2D_test_445836_0.50000..pkl
NavierStokes2D_test_471851_0.50000.pkl NavierStokes2D_test_471851_0.50000..pkl
NavierStokes2D_train_403221_0.50000_100.pkl NavierStokes2D_train_403221_0.50000_100..pkl
NavierStokes2D_test_585798_0.50000.pkl NavierStokes2D_test_585798_0.50000..pkl
NavierStokes2D_train_954413_0.50000_100.pkl NavierStokes2D_train_954413_0.50000_100..pkl
NavierS

100%|██████████| 310/310 [22:58<00:00,  4.45s/it] 

NavierStokes2D_valid_234561_0.50000.pkl NavierStokes2D_valid_234561_0.50000..pkl
NavierStokes2D_test_857666_0.50000.pkl NavierStokes2D_test_857666_0.50000..pkl
NavierStokes2D_test_950847_0.50000.pkl NavierStokes2D_test_950847_0.50000..pkl
NavierStokes2D_train_857466_0.50000_100.pkl NavierStokes2D_train_857466_0.50000_100..pkl
NavierStokes2D_train_516076_0.50000_100.pkl NavierStokes2D_train_516076_0.50000_100..pkl
NavierStokes2D_train_452245_0.50000_100.pkl NavierStokes2D_train_452245_0.50000_100..pkl
NavierStokes2D_valid_61849_0.50000.pkl NavierStokes2D_valid_61849_0.50000..pkl
NavierStokes2D_train_975533_0.50000_100.pkl NavierStokes2D_train_975533_0.50000_100..pkl
NavierStokes2D_test_666797_0.50000.pkl NavierStokes2D_test_666797_0.50000..pkl
NavierStokes2D_valid_144407_0.50000.pkl NavierStokes2D_valid_144407_0.50000..pkl
NavierStokes2D_test_588619_0.50000.pkl NavierStokes2D_test_588619_0.50000..pkl
NavierStokes2D_test_337593_0.50000.pkl NavierStokes2D_test_337593_0.50000..pkl
NavierSt